In [ ]:
import plotly.graph_objects as go
import numpy as np

In [ ]:
from dataclasses import dataclass
import numpy as np

@dataclass
class Detector:
    name: str
    x0: float
    y0: float
    z0: float
    dx: np.ndarray    # direction vector along i
    dy: np.ndarray    # direction vector along j
    dz: np.ndarray    # direction vector along i
    i: int            # grid index
    j: int            # grid index
    value: float      # measurement
    n: int = 4            # grid resolution (e.g. 4x4)

size=1 #keep it like this 

# Adding cathode
dx = np.array([size, 0, 0])
dy = np.array([0, size, 0])
dz = np.array([0, 0, 0])

x0, y0, z0 = 0, 0, 1
detectors = {
    "C1": Detector("C1", x0=x0, y0=y0, z0=z0, dx=dx, dy=dy, dz=dz, i=0, j=1, value=500),
    "C2": Detector("C2", x0=x0, y0=y0, z0=z0, dx=dx, dy=dy, dz=dz, i=2, j=0, value=500),
    "C3": Detector("C3", x0=x0, y0=y0, z0=z0, dx=dx, dy=dy, dz=dz, i=3, j=2, value=500),
    "C4": Detector("C4", x0=x0, y0=y0, z0=z0, dx=dx, dy=dy, dz=dz, i=1, j=3, value=500),
}
x0, y0, z0 = 1, 0, 1
detectors.update(
    {
        "C5": Detector("C5", x0=x0, y0=y0, z0=z0, dx=dx, dy=dy, dz=dz, i=0, j=1, value=500),
        "C6": Detector("C6", x0=x0, y0=y0, z0=z0, dx=dx, dy=dy, dz=dz, i=2, j=0, value=500),
        "C7": Detector("C7", x0=x0, y0=y0, z0=z0, dx=dx, dy=dy, dz=dz, i=2, j=2, value=500),
        "C8": Detector("C8", x0=x0, y0=y0, z0=z0, dx=dx, dy=dy, dz=dz, i=1, j=3, value=500),
    }
)

# Adding membrane
dx = np.array([0, 0, 0])
dy = np.array([0, size, 0])
dz = np.array([0, 0, size])

x0, y0, z0 = 0, 0, 1
detectors.update(
    {
        "M1": Detector("M1", x0=x0, y0=y0, z0=z0, dx=dx, dy=dy, dz=dz, i=3, j=1, value=200),
        "M2": Detector("M2", x0=x0, y0=y0, z0=z0, dx=dx, dy=dy, dz=dz, i=1, j=1, value=200),
    }
)
x0, y0, z0 = 0, 0, 0
detectors.update(
    {
        "M5": Detector("M5", x0=x0, y0=y0, z0=z0, dx=dx, dy=dy, dz=dz, i=3, j=1, value=200),
        "M6": Detector("M6", x0=x0, y0=y0, z0=z0, dx=dx, dy=dy, dz=dz, i=1, j=1, value=200),
    }
)


x0, y0, z0 = 2, 0, 1
detectors.update(
    {
        "M3": Detector("M3", x0=x0, y0=y0, z0=z0, dx=dx, dy=dy, dz=dz, i=3, j=1, value=200),
        "M4": Detector("M4", x0=x0, y0=y0, z0=z0, dx=dx, dy=dy, dz=dz, i=1, j=1, value=200),
    }
)

x0, y0, z0 = 2, 0, 0
detectors.update(
    {
        "M7": Detector("M7", x0=x0, y0=y0, z0=z0, dx=dx, dy=dy, dz=dz, i=3, j=1, value=200),
        "M8": Detector("M8", x0=x0, y0=y0, z0=z0, dx=dx, dy=dy, dz=dz, i=1, j=1, value=200),
    }
)


In [ ]:
# ---------------------------------------------------------
# Helper Functions
# ---------------------------------------------------------

def cube_edges(x0, y0, z0, size):
    """Return list of line segments (x,y,z) for cube edges."""
    x = [x0, x0+size]
    y = [y0, y0+size]
    z = [z0, z0+size]

    edges = []
    # 4 bottom
    edges += [([x[0], x[1]], [y[0], y[0]], [z[0], z[0]])]
    edges += [([x[0], x[1]], [y[1], y[1]], [z[0], z[0]])]
    edges += [([x[0], x[0]], [y[0], y[1]], [z[0], z[0]])]
    edges += [([x[1], x[1]], [y[0], y[1]], [z[0], z[0]])]

    # 4 top
    edges += [([x[0], x[1]], [y[0], y[0]], [z[1], z[1]])]
    edges += [([x[0], x[1]], [y[1], y[1]], [z[1], z[1]])]
    edges += [([x[0], x[0]], [y[0], y[1]], [z[1], z[1]])]
    edges += [([x[1], x[1]], [y[0], y[1]], [z[1], z[1]])]

    # vertical
    edges += [([x[0], x[0]], [y[0], y[0]], [z[0], z[1]])]
    edges += [([x[1], x[1]], [y[0], y[0]], [z[0], z[1]])]
    edges += [([x[0], x[0]], [y[1], y[1]], [z[0], z[1]])]
    edges += [([x[1], x[1]], [y[1], y[1]], [z[0], z[1]])]

    return edges


def add_cube(fig, x, y, z, size):
    """Adds cube edges to the figure."""
    for xe, ye, ze in cube_edges(x, y, z, size):
        fig.add_trace(go.Scatter3d(
            x=xe, y=ye, z=ze,
            mode='lines',
            line=dict(color="black", width=4),
            showlegend=False
        ))


def add_grid_face(fig, x0, y0, z0, dx, dy, dz, n=4, color="gray"):
    """
    Adds an n×n grid face defined by vector directions (dx, dy, dz).
    The face lies in plane: (x0,y0,z0) + a*dx + b*dy
    """
    for i in range(n+1):
        # lines parallel to dx
        fig.add_trace(go.Scatter3d(
            x=[x0 + (i/n)*dx[0], x0 + (i/n)*dx[0] + dy[0]],
            y=[y0 + (i/n)*dx[1], y0 + (i/n)*dx[1] + dy[1]],
            z=[z0 + (i/n)*dx[2], z0 + (i/n)*dx[2] + dy[2]],
            mode='lines', line=dict(color=color, width=2),
            showlegend=False
        ))
        # lines parallel to dy
        fig.add_trace(go.Scatter3d(
            x=[x0 + (i/n)*dy[0], x0 + (i/n)*dy[0] + dx[0]],
            y=[y0 + (i/n)*dy[1], y0 + (i/n)*dy[1] + dx[1]],
            z=[z0 + (i/n)*dy[2], z0 + (i/n)*dy[2] + dx[2]],
            mode='lines', line=dict(color=color, width=2),
            showlegend=False
        ))


def add_detector(fig, x0, y0, z0, dx, dy, i, j, n=4, color="red"):
    """Add a filled detector square at grid position (i,j)."""
    # bottom-left of the square
    xA = x0 + (i/n)*dx[0] + (j/n)*dy[0]
    yA = y0 + (i/n)*dx[1] + (j/n)*dy[1]
    zA = z0 + (i/n)*dx[2] + (j/n)*dy[2]

    dxs = (1/n)*dx
    dys = (1/n)*dy

    # corners of the square
    X = [xA,
         xA + dxs[0],
         xA + dxs[0] + dys[0],
         xA + dys[0],
         xA]
    Y = [yA,
         yA + dxs[1],
         yA + dxs[1] + dys[1],
         yA + dys[1],
         yA]
    Z = [zA,
         zA + dxs[2],
         zA + dxs[2] + dys[2],
         zA + dys[2],
         zA]

    fig.add_trace(go.Scatter3d(
        x=X, y=Y, z=Z,
        mode='lines+markers',
        surfaceaxis=2,
        line=dict(color=color, width=5),
        marker=dict(size=2),
        showlegend=False
    ))

def add_detector(fig, x0, y0, z0, dx, dy, dz, i, j, n=4, value=0.0, name="", vmin=0, vmax=1, threshold=2):
    import plotly.colors as pc

    dx = np.array(dx)
    dy = np.array(dy)
    dz = np.array(dz)
    shift_membrane_y = 0 if not dz.any() else 0.125
    shift_membrane_z = 0 if not dz.any() else -0.05

    # Normalize incoming value
    t = (value - vmin) / (vmax - vmin)

    # For log scale instead
    # Avoid log(0) or log of negative values
    eps = 1e-12
    value_clamped = max(value, eps)
    vmin_clamped  = max(vmin,  eps)
    vmax_clamped  = max(vmax,  eps)
    
    # Normalized log scale
    t = (np.log10(value_clamped) - np.log10(vmin_clamped)) / \
        (np.log10(vmax_clamped) - np.log10(vmin_clamped))
    
    # Clamp
    t = min(max(t, 0), 0.999)

    # Correct color selection
    color = pc.sample_colorscale("Jet", t)[0]
    if value < threshold:
        color="grey"


    # Bottom-left corner
    xA = x0 + (i/n)*dx[0] + (j/n)*dy[0] + (i/n)*dz[0]
    yA = y0 + (i/n)*dx[1] + (j/n)*dy[1] + (i/n)*dz[1] + shift_membrane_y
    zA = z0 + (i/n)*dx[2] + (j/n)*dy[2] + (i/n)*dz[2] + shift_membrane_z

    dxs = (1/n)*dx
    dys = (1/n)*dy
    dzs = (1/n)*dz

    # 4 corners of the square
    X = [ xA, xA + dxs[0] + dzs[0], xA + dxs[0] + dys[0] + dzs[0], xA + dys[0] ]
    Y = [ yA, yA + dxs[1] + dzs[1], yA + dxs[1] + dys[1] + dzs[1], yA + dys[1] ]
    Z = [ zA, zA + dxs[2] + dzs[2], zA + dxs[2] + dys[2] + dzs[2], zA + dys[2] ]

    # Two triangles
    fig.add_trace(go.Mesh3d(
        x=X, y=Y, z=Z,
        i=[0], j=[1], k=[2],
        color=color, opacity=0.9,
        flatshading=True,
        showscale=False,
    ))
    fig.add_trace(go.Mesh3d(
        x=X, y=Y, z=Z,
        i=[0], j=[2], k=[3],
        color=color, opacity=0.9,
        flatshading=True,
        showscale=False,
    ))
    xc = sum(X)/4
    yc = sum(Y)/4
    zc = sum(Z)/4
    fig.add_trace(go.Scatter3d(
        x=[xc+0.02], y=[yc], z=[zc + 0.02],   # slightly above the detector
        mode='text',
        text=[name],
        textposition='middle center',
        textfont=dict(color="white"),
        showlegend=False
    ))
def add_detector_obj(fig, det: Detector, vmin=0, vmax=1000, threshold=400): add_detector(fig, det.x0, det.y0, det.z0, det.dx, det.dy, det.dz, det.i, det.j, det.n, det.value, det.name, vmin=vmin, vmax=vmax, threshold=threshold)


In [ ]:
import pandas as pd
dffull = pd.read_csv("detector_responses.csv")

In [ ]:
df = dffull.pivot(index="mask", columns="module", values="value")
df

In [ ]:

def draw_detector_viewer(df:pd.DataFrame, mask=1, threshold=2):
    # ---------------------------------------------------------
    # Build Scene
    # ---------------------------------------------------------
    
    size = 1  # cube size
    fig = go.Figure()
    
    # Bottom cubes
    add_cube(fig, 0, 0, 0, size)
    add_cube(fig, 1, 0, 0, size)
    
    # Top cubes
    add_cube(fig, 0, 0, 1, size)
    add_cube(fig, 1, 0, 1, size)
    
    # ---------------------------------------------------------
    # Grid on the interface between bottom and top cubes
    # Face is the top face of bottom-left cube
    # ---------------------------------------------------------
    
    # Grid vectors
    dx = np.array([size, 0, 0])
    dy = np.array([0, size, 0])
    dz = np.array([0, 0, 0])
    
    # bottom-left top face origin
    x0, y0, z0 = 0, 0, 1
    add_grid_face(fig, x0, y0, z0, dx, dy, (0,1,0), n=4)
    
    # bottom-right top face origin
    x0, y0, z0 = 1, 0, 1
    add_grid_face(fig, x0, y0, z0, dx, dy, (0,1,0), n=4)
    
    vmin = df.loc[mask].min() 
    vmax = df.loc[mask].max() 
    for dname, det in detectors.items():
        det.value = df.loc[mask, dname]
        if np.isnan(det.value):
            det.value = min(vmin, threshold)
        add_detector_obj(fig, det, vmin=vmin, vmax=vmax, threshold=threshold)
    
    
    # ---------------------------------------------------------
    # Axes settings
    # ---------------------------------------------------------
    
    fig.update_layout(
        width=800,
        height=800,
        title=dict(text=f"Mask: {mask}", font=dict(size=25), automargin=True, yref='paper'),
        scene=dict(
            xaxis=dict(visible=False),
            yaxis=dict(visible=False),
            zaxis=dict(visible=False),
            annotations=[
                dict(
                    showarrow=False,
                    x=-0.5, y=0.5, z=0.5,
                    text="non-TCO",
                    font=dict(size=16, color="black"),
                    xanchor="right"
                ),
                dict(
                    showarrow=False,
                    x=2.5, y=0.5, z=0.5,
                    text="TCO",
                    font=dict(size=16, color="black"),
                    xanchor="left"
                )
            ]
        ),
        scene_camera=dict(eye=dict(x=1, y=-1.8, z=1.2)),
    )

    # Values for colorbar ticks
    log_min = np.log10(vmin)
    log_max = np.log10(vmax)
    log_mid = (log_min + log_max)/2
    
    fig.add_trace(go.Scatter3d(
        x=[None], y=[None], z=[None],   # completely invisible
        mode="markers",
        marker=dict(
            size=0.1,
            color=[log_min, log_max],   # log range!
            colorscale="Jet",
            cmin=log_min,
            cmax=log_max,
            showscale=True,
            colorbar=dict(
                title="Mean PEs",
                thickness=20,
                len=0.7,
                tickvals=[log_min, log_mid, log_max],
                ticktext=[f"{vmin:.3g}", 
                          f"{10**log_mid:.3g}", 
                          f"{vmax:.3g}"],
            ),
        ),
        showlegend=False
    ))
    
    fig.show()

In [ ]:
draw_detector_viewer(df, mask=16, threshold = 1.5)